<a href="https://colab.research.google.com/github/livjab/food-desert-DS/blob/master/Food_Deserts_Final2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import re
import numpy as np

In [3]:
# Specify path and load the csv file
path = '/Users/Dustin/Desktop/Food Deserts/csv_files/'

business_csv = path + 'Listing_of_Active_Businesses.csv'

pd.set_option('display.max_columns', 500)

df_business = pd.read_csv(business_csv)


FileNotFoundError: ignored

In [0]:
# Return a dataframe that contains market or food.
df_business_market = df_business[df_business['PRIMARY NAICS DESCRIPTION'].str.contains('market|grocery', flags=re.IGNORECASE, regex=True) == True]

In [0]:
# Return a dataframe that doesn't contain 'telephone|business|research'
df_business_market_clean = df_business_market[~df_business_market['PRIMARY NAICS DESCRIPTION'].str.contains('telephone|business|research') == True]

In [0]:
# Extract Latitude and put it in a column
df_business_market_clean['LATITUDE'] = df_business_market_clean['LOCATION'].str.extract('\((.*),').astype(float)

C:\Users\Dustin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
# Extract Longitude and put it in a column
df_business_market_clean['LONGITUDE'] = df_business_market_clean['LOCATION'].str.extract(',(.*)\)').astype(float)

C:\Users\Dustin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
# Remove coordinates outside of LA
df_business_market_clean = df_business_market_clean[(df_business_market_clean['LONGITUDE'] < -117.7) & (df_business_market_clean['LONGITUDE'] > -118.9)]

In [0]:
# Dataframe with no Nulls for location
df_test = df_business_market_clean.dropna(subset=['LOCATION'])

In [0]:
# Export
# df_test.to_csv('clean.csv', index=False)

# Change to a different Map Projection

In [0]:
import pyproj

project_projection = pyproj.Proj("+init=EPSG:3857") # Output map projections
google_projection = pyproj.Proj("+init=EPSG:4326") # Input map projections

# Save Coordinates to a list
longitude = df_test['LONGITUDE'].values
latitude = df_test['LATITUDE'].values

# Output mercator map projections where x=longitude and y=latitude
x, y = pyproj.transform(google_projection, project_projection, longitude, latitude)

# Bokeh Plot

In [0]:
from bokeh.plotting import figure, show, output_file
from bokeh.tile_providers import get_provider, Vendors
from bokeh.models import ColumnDataSource

In [0]:
# HTML file
output_file('market_location.html')
# Name of tile used for plotting
tile = get_provider(Vendors.STAMEN_TERRAIN)

# Instantiate the figure
p = figure(plot_width=1000, plot_height=1000, x_axis_type = "mercator", y_axis_type = "mercator")

# Define the data to a dictionary
source = ColumnDataSource(data = dict(lat=y,lon=x))

# Add the tile/map
p.add_tile(tile)
# Plot it to the tile
p.circle(x='lon', y='lat', source=source, alpha=0.1)

# Display the map
show(p)

# Need to clean-up longitiude and latitude that are not in Los Angeles

# GEOPY - Get the distance of 'n' miles

In [0]:
import geopy
import geopy.distance

def travel(lat, lon, bearing=0, miles=0.5):
    # Starting point
    start = geopy.Point(lat, lon)

    # Initialized with a distance of 2 mi.
    d = geopy.distance.geodesic(miles=miles)

    # Destination method which takes the starting point and bearings(North=0).
    # Return coordinates from start to distance,d, traveled
    coord = d.destination(point=start, bearing=bearing)
    return coord
    # printing(coord) gives degrees, min, and sec
    # Slice it coord gets you google mapping

# SHAPELY - Make a circle around the target and the groceries within the circle

In [0]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [0]:
def make_circle(lat, lon, miles=0.5):
    circle_points = []  
    for i in range(360):
        new_point = travel(lat, lon, bearing=i, miles=miles)
        new_point = new_point[0], new_point[1]
        circle_points.append(new_point)
    circle = Polygon(circle_points)
    return circle

In [0]:
def get_groceries_within_circle(df, circle):
    df_test_copy = df.copy()
    in_circle = []
    for i in range(len(df_test_copy)):
        lat = df_test_copy.iloc[i]['LATITUDE']
        lon = df_test_copy.iloc[i]['LONGITUDE']
        point = Point(lat, lon)
        
        if circle.contains(point):
            # In Circle
            in_circle.append(True)
        else:
            # Not In Circle
            in_circle.append(False)

    df_test_copy['in_circle'] = in_circle

    # Returns a data frame containing all grocery stores within the circle
    return df_test_copy[df_test_copy['in_circle'] == True]

In [0]:
circle = make_circle(34.0401, -118.2106, 0.25)

In [0]:
get_groceries_within_circle(df_test, circle).shape

(5, 19)